In [1]:
import multiprocessing
import os
import pickle
import sys
import typing
import uuid
import zipfile
import gc
from collections import Counter

import cv2
import numpy as np
import pandas as pd
import scipy
import torch
import torch.nn.functional as F
import wandb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torchvision.models import resnet
from tqdm.auto import tqdm

tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger(__name__)
logger.setLevel('ERROR')
import matplotlib.pyplot as plt
logger = logging.getLogger('matplotlib')
logger.setLevel('ERROR')
import PIL
from PIL import Image
logger = logging.getLogger('PIL')
logger.setLevel('ERROR')


print(sys.version_info)
print(torch.__version__)
print(torch.cuda.is_available())

sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)
1.12.1+cu102
False


In [2]:
torch.set_num_threads(48)
torch.set_num_interop_threads(48)

In [3]:
root = "/home/asciishell/s3/jupyter.asciishell.ru"

In [4]:
df = pd.read_csv(f"{root}/train.csv")[["id", "glasses"]].copy()

In [5]:
df_train, df_valid = train_test_split(df, test_size=0.3, random_state=42)
df_valid, df_test = train_test_split(df_valid, test_size=0.33, random_state=42)

In [6]:
df_train.shape, df_valid.shape, df_test.shape

((3150, 2), (904, 2), (446, 2))

In [7]:
train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]),
    ]
)


class FaseDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        root: str,
        size,
        sample: pd.DataFrame,
        transform: typing.Callable,
    ):
        self.root = root
        self.size = size
        self.ids = sample["id"].values
        self.targets = sample["glasses"].values
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        img, target = self.ids[index], self.targets[index]
        img = Image.open(self.root.format(img))
        img = img.resize(self.size, Image.Resampling.BILINEAR)

        pos_1 = self.transform(img)
        pos_2 = self.transform(img)

        return pos_1, pos_2, target

In [8]:
im_size = (128, 128)
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    FaseDataset(f"{root}/faces-spring-2020/faces-spring-2020/face-{{}}.png", im_size, df_train, train_transform),
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
)
test_loader = torch.utils.data.DataLoader(
    FaseDataset(f"{root}/faces-spring-2020/faces-spring-2020/face-{{}}.png", im_size, df_valid, test_transform),
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
)

In [9]:
from torch import nn
from torchvision import models

class WideResnet50(nn.Module):
    def __init__(self, num_classes, use_pre_trained=True):
        super().__init__()

        model = models.wide_resnet50_2(use_pre_trained)
        self.num_classes = num_classes
        self.features = nn.Sequential(*list(model.children())[:-1])
        in_features = model.fc.in_features
        self.classifier = nn.Sequential(
            nn.Linear(in_features, num_classes, bias=True)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.features(x)

        out = torch.flatten(x, 1)

        x = self.classifier(out)
        if self.num_classes == 1:
            x = self.sigmoid(x)

        return F.normalize(out, dim=-1), x


class ContrastiveLoss(torch.nn.Module):
    def __init__(self, temperature, cuda):
        super().__init__()
        self.temperature = temperature
        self.cuda = cuda

    def get_negative_mask(self, batch_size):
        negative_mask = torch.ones((batch_size, 2 * batch_size), dtype=bool)
        for i in range(batch_size):
            negative_mask[i, i] = 0
            negative_mask[i, i + batch_size] = 0

        negative_mask = torch.cat((negative_mask, negative_mask), 0)
        return negative_mask

    def forward(self, out_1, out_2):
        batch_size = out_1.shape[0]

        # neg score
        out = torch.cat([out_1, out_2], dim=0)
        neg = torch.exp(torch.mm(out, out.t().contiguous()) / self.temperature)
        mask = self.get_negative_mask(batch_size)
        if self.cuda:
            mask = mask.cuda()
        neg = neg.masked_select(mask).view(2 * batch_size, -1)

        # pos score
        pos = torch.exp(torch.sum(out_1 * out_2, dim=-1) / self.temperature)
        pos = torch.cat([pos, pos], dim=0)

        # estimator g()
        Ng = neg.sum(dim=-1)

        # contrastive loss
        loss = (-torch.log(pos / (pos + Ng))).mean()

        return loss

In [10]:
def main(
    *,
    model,
    criterion_cls,
    criterion_contrastive,
    optimizer,
    writer,
    train_loader,
    valid_loader,
    checkpoint_path,
    cuda=True,
    epochs=200,
    alpha=0.1,
):
    if cuda:
        model = model.cuda()
    model = torch.nn.DataParallel(model)
    step = 0
    for epoch in range(1, epochs + 1):
        epoch_alpha = alpha + (1.0 - alpha) * (epoch / epochs)
        writer.add_scalar("train/alpha", epoch_alpha, epoch)
        model.train()
        total_loss, total_num = 0.0, 0
        for pos_1, pos_2, target in tqdm(train_loader, desc=f"Train {epoch}"):
            if cuda:
                pos_1 = pos_1.cuda(non_blocking=True)
                pos_2 = pos_2.cuda(non_blocking=True)
                target = target.cuda(non_blocking=True)
            out_1, out_1_proba = model(pos_1)
            out_2, out_2_proba = model(pos_2)

            loss = epoch_alpha * (criterion_cls(out_1_proba, target) + criterion_cls(out_2_proba, target)) + (
                1.0 - epoch_alpha
            ) * criterion_contrastive(out_1, out_2)
            writer.add_scalar("loss/train", loss, step)
            step += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_num += batch_size
            total_loss += loss.item() * batch_size

        train_loss = total_loss / total_num

        if epoch % 1 == 0:
            with torch.no_grad():
                model.eval()
                predicts = []
                targets = []
                for pos_1, pos_2, target in tqdm(valid_loader, desc=f"Valid {epoch}"):
                    if cuda:
                        pos_1 = pos_1.cuda(non_blocking=True)
                    _, predict = model(pos_1)
                    predicts.extend(predict.cpu().numpy())
                    targets.extend(target.cpu().numpy())
            predicts = np.stack(predicts)
            targets = np.stack(targets)
            writer.add_scalar("valid/f1w", f1_score(targets, predicts.argmax(axis=1), average="weighted"), epoch)
            writer.add_scalar("valid/acc", accuracy_score(targets, predicts.argmax(axis=1)), epoch)
            writer.add_scalar(
                "valid/roc_auc",
                roc_auc_score(targets, scipy.special.softmax(predicts, axis=1)[:, 1]),
                epoch,
            )
            del predicts, targets
            torch.save(model.state_dict(), checkpoint_path.format(epoch))
        writer.flush()
        gc.collect()
    writer.close()

In [11]:
def main3():
    cuda = False
    epochs = 10
    model_arch = "wideresnet"
    init = True
    cls_w = 0.5
    run = f"glass-{model_arch}-init={init}-cls_w={cls_w}-bs={batch_size}"
    os.makedirs(run, exist_ok=True)
    wandb.init(
        project="glasses_cv",
        dir=run,
        name=run,
        config={
            "model": model_arch,
            "init": init,
            "classification_weight": cls_w,
            "batch_size": batch_size,
            "epochs": epochs,
        },
    )
    wandb.tensorboard.patch(root_logdir=run)
    writer = SummaryWriter(run)
    try:
        model = WideResnet50(2, init)
        criterion_cls = torch.nn.CrossEntropyLoss()
        criterion_contrastive = ContrastiveLoss(0.5, cuda)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)

        print("Start", run)
        main(
            model=model,
            criterion_cls=criterion_cls,
            criterion_contrastive=criterion_contrastive,
            optimizer=optimizer,
            writer=writer,
            train_loader=train_loader,
            valid_loader=test_loader,
            checkpoint_path=f"{run}/model_{{}}.pt",
            cuda=cuda,
            epochs=epochs,
            alpha=cls_w,
        )
    finally:
        writer.close()
        wandb.finish()
        wandb.tensorboard.unpatch()


if __name__ == "__main__":
    main3()

wandb: Currently logged in as: asciishell (mlao). Use `wandb login --relogin` to force relogin


2022-12-15 14:43:09.216318: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 14:43:13.977811: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-15 14:43:16.148566: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/asciishell/project/venv3.10/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-15 14:43

Start glass-wideresnet-init=True-cls_w=0.5-bs=128


Train 1:   0%|          | 0/25 [00:00<?, ?it/s]

Valid 1:   0%|          | 0/8 [00:00<?, ?it/s]

...

Train 9:   0%|          | 0/25 [00:00<?, ?it/s]

Valid 9:   0%|          | 0/8 [00:00<?, ?it/s]

Train 10:   0%|          | 0/25 [00:00<?, ?it/s]

Valid 10:   0%|          | 0/8 [00:00<?, ?it/s]

global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/train,█▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁
train/alpha,▁▂▃▃▄▅▆▆▇█
valid/acc,▆▅█▁█▆▇▆▅▆
valid/f1w,▆▅█▁█▆█▆▅▆
valid/roc_auc,▃▄▅▅▅▅█▅▁▃
global_step,10
loss/train,0.42184
train/alpha,1.0
valid/acc,0.8927
valid/f1w,0.89133


In [13]:
model = WideResnet50(2, False)
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load('glass-wideresnet-init=True-cls_w=0.5-bs=128/model_7.pt'))
model.eval();

In [14]:
oos_loader = torch.utils.data.DataLoader(
    FaseDataset(f"{root}/faces-spring-2020/faces-spring-2020/face-{{}}.png", im_size, df_test, test_transform),
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
)

In [15]:
cuda = False
with torch.no_grad():
    model.eval()
    predicts = []
    targets = []
    for pos_1, pos_2, target in tqdm(oos_loader, desc=f"Test"):
        if cuda:
            pos_1 = pos_1.cuda(non_blocking=True)
        _, predict = model(pos_1)
        predicts.extend(predict.cpu().numpy())
        targets.extend(target.cpu().numpy())
predicts = np.stack(predicts)
targets = np.stack(targets)
print("f1", f1_score(targets, predicts.argmax(axis=1), average=None))
print("f1 macro", f1_score(targets, predicts.argmax(axis=1), average='macro'))
print("f1 weighted", f1_score(targets, predicts.argmax(axis=1), average='weighted'))

print("acc", accuracy_score(targets, predicts.argmax(axis=1)))
print("roc_auc", roc_auc_score(targets, scipy.special.softmax(predicts, axis=1)[:, 1]))

Test:   0%|          | 0/4 [00:00<?, ?it/s]

f1 [0.85043988 0.90744102]
f1 macro 0.8789404495159427
f1 weighted 0.886736568690372
acc 0.8856502242152466
roc_auc 0.9649191444966093
